In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime
# from ./src/commissioningschedule/scheduler import gather_task_xmls, merge_schedules
# import src/commissioningschedule/scheduler as scheduler
from scheduler import gather_task_xmls, merge_schedules

import xml.etree.ElementTree as ET
import os
import re

from astropy.time import Time

/Users/bhord/anaconda3/envs/pandora/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [3]:
# xml_dir = "/Users/bhord/research/pandora/commissioning_scheduler/v1_09092025/"
xml_dir = "/Users/bhord/research/pandora/commissioning_scheduler/v2_09192025/"
output_path = "./master_schedule.xml"
cvz_coords = (120.0, 8.0)

In [4]:
# Updated TLE from MOC July 30, 2025
tle1 = "1 99152U 26011B   26005.66013674 +.00000000 +00000-0 +00000-0 0    16"
tle2 = "2 99152  97.6750  17.6690 0000000 328.8990  20.9640 14.86530781000004"

In [8]:
xml_paths = gather_task_xmls(xml_dir)

In [9]:
from scheduler import gather_task_xmls, merge_schedules

result = merge_schedules(
    xml_paths,
    output_path,
    cvz_coords,
    tle1, tle2,
    dependency_json="dependency.json",
    # progress_json="progress.json",
    # extra_cvz_json="extra_cvz.json",
    pointing_ephem_file="/Users/bhord/research/pandora/soc/Pandora-600km-withdrag-20250820.txt",
    commissioning_start=datetime(2026, 1, 15, 0, 0),  # 10 days for bus commissioning
    commissioning_end=datetime(2026, 2, 5, 0, 0)
)

Skipping task 0342 observation 0342_000_template_SOC.xml until closer to full moon (current: 2026-01-15, target: 2026-02-02)
Processing task 0310: 0310_000_SOC.xml, RA: 123.45670577640334, Dec: -5.6666666666667105
Observation needs 0.7 minutes
All observations for task 0310 completed
Processing task 0312: 0312_000_SOC.xml, RA: 118.985495, Dec: 5.825131
Task 0312 total observation time: 6000 seconds (100.0 minutes)
Scheduled Task 0312 data sequence: 300s
Scheduled Task 0312 stare sequence: 900s
Scheduled Task 0312 data sequence: 300s
Scheduled Task 0312 stare sequence: 900s
Scheduled Task 0312 data sequence: 300s
Scheduled Task 0312 stare sequence: 900s
Scheduled Task 0312 data sequence: 300s
Scheduled Task 0312 stare sequence: 900s
Scheduled Task 0312 data sequence: 300s
Scheduled Task 0312 stare sequence: 900s
All observations for task 0312 completed
Processing task 0315: 0315_000_SOC.xml, RA: 176.729457161733, Dec: 27.0395369695943
Observation needs 2.0 minutes
Processing task 0315: 

In [10]:
from scheduler import analyze_schedule_diagnostics, print_diagnostics_report

diagnostics = analyze_schedule_diagnostics(
    'master_schedule.xml',
    commissioning_end=Time(datetime(2026, 2, 4, 23, 59, 59), format='datetime', scale='utc').datetime,
    input_xml_dir=xml_dir,
    debug=True,
    detailed_sequences=True,
)
# diagnostics = analyze_schedule_diagnostics('master_schedule.xml', commissioning_end=datetime(2026, 2, 4, 23, 59, 59))
print_diagnostics_report(diagnostics)

# For CSV output with all details
# print_diagnostics_report(diagnostics, output_format="csv", output_file="detailed_schedule.csv")

Building task mappings from input XML files...
  Task 0343: Target 'Sirius' at RA=101.42588788888888, DEC=-16.715742
  Task 0343: Target 'Sirius' at RA=101.17588788888888, DEC=-16.715742
  Task 0315: Target 'G4017930742788615552' at RA=176.729457161733, DEC=27.072816969594303
  Task 0315: Target 'G4017930742788615552' at RA=176.66289716173299, DEC=26.972976969594303
  Task 0318: Target 'G3143682556189212672' at RA=118.985495, DEC=5.825131
  Task 0360: Target 'U_Crt' at RA=168.18876369352, DEC=-7.29848919311
  Task 0341: Target 'GJ_3470' at RA=119.77433139863999, DEC=15.3914544625
  Task 0319: Target 'pnt174_247198-016_830752' at RA=174.2471975, DEC=-16.8307524
  Task 0330: Target 'G3547915223000134144' at RA=174.1188423654, DEC=-16.8504849173
  Task 0330: Target 'G3547915223000134144' at RA=174.1188423654, DEC=-16.8504849173
  Task 0342: Target 'GJ_3470' at RA=119.77433139863999, DEC=15.3914544625
  Task 0319: Target 'pnt174_247198-016_830752' at RA=174.2471975, DEC=-16.8307524
  Task 

In [35]:
path = "/Users/bhord/research/pandora/commissioning_scheduler/v1_09092025/0310_000_SOC.xml"
tree = ET.parse(path)
root = tree.getroot()
NS = {"cal": "/pandora/calendar/"}
print(root)
fn = os.path.basename(path)
m = re.match(r"(\d{4})_(\d{3})_.*\.xml", fn)
visit_id, obs_id = m.group(1), m.group(2)
tgt_elem = root.find('.//cal:Observational_Parameters/cal:Target', namespaces=NS)
# tgt_elem = root.findall('ScienceCalendar/Visit/Observation_Sequence/Observational_Parameters/Target')
print(tgt_elem)
print(root.tag)
for child in root:
    print(child.tag)
print(tree)

<Element '{/pandora/calendar/}ScienceCalendar' at 0x7fc92f6037e0>
<Element '{/pandora/calendar/}Target' at 0x7fc92f602480>
{/pandora/calendar/}ScienceCalendar
{/pandora/calendar/}Meta
{/pandora/calendar/}Visit


In [72]:
result

{'total_bytes': 0, 'downlinks_required': 0, 'visits_written': 0}